In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[28]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","32g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 22:37:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[28] appName=Create-DataFrame>

In [2]:
!wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

--2025-01-28 22:37:27--  https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip
128.195.10.252ive.ics.uci.edu (archive.ics.uci.edu)... 
connected. to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... 
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘recipe+reviews+and+user+feedback+dataset.zip’

recipe+reviews+and+     [        <=>         ]   2.02M   294KB/s    in 6.6s    

2025-01-28 22:37:34 (314 KB/s) - ‘recipe+reviews+and+user+feedback+dataset.zip’ saved [2114088]



In [3]:
!ls

lab05.ipynb		       pan-tadeusz.txt
lab06.ipynb		       polish.stopwords.txt
lab5.ipynb		       recipe+reviews+and+user+feedback+dataset.zip
pan_tadeusz_bag_of_words.json  venv


In [4]:
!mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

In [5]:


# wypakowujemy plik do podfolderu data
import zipfile
with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")



In [6]:
!ls ./data

'Recipe Reviews and User Feedback Dataset.csv'


In [7]:
# sprawdzamy jak wyglądają 3 pierwsze linie pliku, widać, że pierwsza zawiera nagłówki kolumn a dane są oddzielone przecinkiem
!head -3 "data/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. I’ve written to them and asked them to please!, bring them back"


In [8]:
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")

In [9]:
# najpopularniejsza metoda ich pobrania to show(), ale jest ich więcej
df_reviews.show(5)

+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

25/01/28 22:38:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [10]:
# rzut oka na schemę tego DataFrame
df_reviews.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



In [11]:
# widać, że wszystkie kolumny są typu string, to jest domyślny sposób wczytywania danych przez spark z plain text
# możemy jednak przekazać dodatkowy parametr, który na podstawie próbki danych spróbuje dobrać typ danych odpowiedni dla kolumny
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",", inferSchema=True)

In [12]:
# po wypisaniu schemy widać zmianę
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- thumbs_up: integer (nullable = true)
 |-- thumbs_down: integer (nullable = true)
 |-- stars: integer (nullable = true)
 |-- best_score: integer (nullable = true)
 |-- text: string (nullable = true)



In [13]:


# ramkę możemy również inicjalizować wskazując pożądane typy danych
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType

data = [("James","","Smith",36636,"M",3000),
    ("Michael","Rose","",40288,"M",4000),
    ("Robert","","Williams",42114,"M",4000),
    ("Maria","Anne","Jones",39192,"F",4000),
    ("Jen","Mary","Brown","","F",1000)
  ]

schema = StructType([ \
    StructField("firstname", StringType(), True), \
    StructField("user_id", StringType(), True), \
    StructField("lastname", StringType(), True), \
    StructField("id", StringType(), True), \
    # błąd konwersji "" na int!
    # StructField("id", LongType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True)
    # chcielibyśmy tak, ale tutaj nie da się za bardzo - błąd konwersji int na decimal!
    # StructField("salary", DecimalType(10,2), True) \
  ])

df_test = spark.createDataFrame(data=data,schema=schema)
df_test.printSchema()



root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [14]:
# możemy wykonać rzutowanie po wczytaniu danych z większością kolumn typu tekstowego
import pyspark.sql.functions as F

df_test = df_test.withColumn("salary", F.col("salary").cast("decimal(10,2)"))
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [15]:
# wyświetlenie danych z pojedynczej kolumny
df_test.select(df_test.salary).show()

+-------+
| salary|
+-------+
|3000.00|
|4000.00|
|4000.00|
|4000.00|
|1000.00|
+-------+



In [16]:
# ile wierszy w ramce?
df_reviews.count()


18268

In [17]:
# DataFrame składa się z obiektów typu Column dla każdej kolumny
# API dla typu Column: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html

# do kolumn możemy się odwoływać tak jak w pandas API, ale wynik jest inny
df_reviews.user_name, df_reviews['user_name']


(Column<'user_name'>, Column<'user_name'>)

In [18]:
# aby wyświetlić dane musimy wywoałać funkcję select na obiekcie dataframe

df_reviews.select(df_reviews.user_name).show(5)

+----------+
| user_name|
+----------+
|   Jeri326|
|   Mark467|
|Barbara566|
|jeansch123|
|  camper77|
+----------+
only showing top 5 rows



In [19]:
# do funkcji select możemy przekazać wiele kolumn a wywołania podobnie jak dla RDD są leniwe
print(df_reviews.select(df_reviews.user_name, df_reviews.user_reputation))
# musimy więc wywołać funkcję, której wykonanie "zmusi" Sparka do wyliczenia jej wartości lub jawnie wywołać np. show
df_reviews.select(df_reviews.user_name, df_reviews.user_reputation).show(5)

DataFrame[user_name: string, user_reputation: string]
+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [20]:


# lub indeksując kolumny innym sposobem
df_reviews.select(df_reviews['user_name'],df_reviews['user_reputation']).show(5)



+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [21]:
from pyspark.sql.functions import isnan, when, count, col

# policzymy teraz liczbę wartości NULL w każdej kolumnie
df_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews.columns]).show()

25/01/28 22:40:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|           45|         64|         74|        77|     80|       83|             84|        86|         86|       86|         86|   86|        86|  86|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



In [22]:
# rzućmy okiem na kilka wierszy gdzie w kolumnie recipe_name jest wartość NULL
df_reviews.filter(df_reviews.recipe_code.isNull()).show()

+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|                 _c0|       recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|      Thank you!!!!"|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
| It was excellent!  |                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
|The recipe was a ...|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|    

25/01/28 22:40:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [23]:
# zapisanie do nowej ramki danych bez wartości pustych
df_reviews_clean = df_reviews.na.drop()
df_reviews_clean.count()

25/01/28 22:40:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


18182

In [25]:
# dla pewności możemy to sprawdzić raz jeszcze
df_reviews_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews_clean.columns]).show()

25/01/28 22:41:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|            0|          0|          0|         0|      0|        0|              0|         0|          0|        0|          0|    0|         0|   0|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



In [26]:
# filtrowanie danych z ramki
df_reviews_clean.filter(df_reviews.user_name.startswith('a')).select(df_reviews_clean.user_name).show(10)
df_reviews_clean.filter(df_reviews.stars == 5).show(10)

+--------------+
|     user_name|
+--------------+
|         ahmom|
|  annamossburg|
|   astarzynski|
|     adamscook|
|      angela32|
|       annaf27|
|    avanhaasen|
|    aunt ann's|
|     angelic0w|
|a_n_g_e_l_0715|
+--------------+
only showing top 10 rows

+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id|       user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|         Jeri326|              1|16656198

25/01/28 22:41:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv
25/01/28 22:41:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expecte

In [27]:
from pyspark.sql.functions import avg

# wyliczenie średniej wartości z kolumny
df_reviews_clean.select(avg(df_reviews_clean.thumbs_up)).show()

+------------------+
|    avg(thumbs_up)|
+------------------+
|1.0892641073589264|
+------------------+



25/01/28 22:41:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [28]:
# ale możemy się dowiedzieć tego i więcej w sposób podobny do tego z biblioteki pandas
df_reviews_clean.select(df_reviews_clean.thumbs_up).describe().show()

+-------+------------------+
|summary|         thumbs_up|
+-------+------------------+
|  count|             18182|
|   mean|1.0892641073589264|
| stddev| 4.201003572820717|
|    min|                 0|
|    max|               106|
+-------+------------------+



25/01/28 22:41:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [29]:
from pyspark.sql.functions import desc
df_reviews_clean.groupby('recipe_code').agg({'thumbs_down': 'sum'}).sort(desc('sum(thumbs_down)')).show(10)

+-----------+----------------+
|recipe_code|sum(thumbs_down)|
+-----------+----------------+
|       2832|             488|
|       9739|             354|
|      17826|             328|
|      18345|             313|
|      12003|             306|
|       4383|             301|
|      41095|             275|
|       8202|             272|
|       6504|             264|
|       6086|             262|
+-----------+----------------+
only showing top 10 rows



25/01/28 22:41:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [31]:
# deklaracja zbiorów wartości dla poszczególnych kolumn przyszłego zbioru danych
header = ['id', 'firstname', 'lastname', 'age', 'salary']
firstnames = ['Adam', 'Katarzyna', 'Krzysztof', 'Marek', 'Aleksandra', 'Zbigniew', 'Wojciech', 'Mieczysław', 'Agata', 'Wisława']
lastnames = ['Mieczykowski', 'Kowalski', 'Malinowski' , 'Szczaw', 'Glut', 'Barański', 'Brzęczyszczykiewicz', 'Wróblewski', 'Wlotka', 'Pysla']
age = {'min': 18, 'max': 68}
salary = {'min': 3200, 'max': 12500}

In [49]:
import random
from tqdm import tqdm

def build_dataset(filename, n_rows=100, chunk_size=100000):
    rows = []
    rows.append(header)
    mu = (salary['max'] + salary['min']) / 2
    sigma = 1000

    with open(filename, 'w', encoding='utf-8') as filehandler:
        
        for id in tqdm(range(1, n_rows + 1), total=n_rows, desc="Building dataset..."):
            row = [
                f'{id}', 
                f'{random.choice(firstnames)}', 
                f'{random.choice(lastnames)}', 
                f"{random.randint(age['min'], age['max'])}",
                f"{round(float(random.normalvariate(mu=mu, sigma=sigma)), 2)}"
            ]
            rows.append(row)
            if id % chunk_size == 0:
                filehandler.writelines([f"{','.join(row)}\n" for row in rows])
                rows = []

In [50]:
# około 715MB zostanie zapisanych w pliku csv, dostosuj ilość rekordów do swoich potrzeb
build_dataset('employee.csv', 20_000_000)

Building dataset...: 100%|███████████████████| 20000000/20000000 [01:28<00:00, 224836.13it/s]


In [51]:
%%time
# więcej magicznych metod w Jupyter Notebooku: https://ipython.readthedocs.io/en/stable/interactive/magics.html
# wczytanie pliku csv przez spark
# df = spark.read.csv('employee.csv', header=True)
df = spark.read.csv('employee.csv', header=True, inferSchema=True)

[Stage 50:==========>                                             (5 + 23) / 28]

CPU times: user 7.8 ms, sys: 5.05 ms, total: 12.9 ms
Wall time: 2.34 s


In [52]:
type(df)

pyspark.sql.dataframe.DataFrame

In [53]:
%%time
# wypisujemy schemat i 10 pierwszych wierszy utworzonego obiektu Spark DataFrame
df.printSchema()
df.show(10)

# przykład wykorzystania funkcji transform, która mapuje wykonanie stworzonej funkcji tu_upper_str_columns na istniejącą kolumnę
# i zwraca nową ramkę z dodatkową kolumną

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)

+---+----------+-------------------+---+-------+
| id| firstname|           lastname|age| salary|
+---+----------+-------------------+---+-------+
|  1| Krzysztof|       Mieczykowski| 31|7997.69|
|  2|  Zbigniew|         Malinowski| 26|7493.97|
|  3| Krzysztof|             Wlotka| 19|9745.33|
|  4|     Agata|Brzęczyszczykiewicz| 43|7968.02|
|  5| Krzysztof|             Szczaw| 39|8192.67|
|  6|   Wisława|               Glut| 35|8441.91|
|  7|      Adam|Brzęczyszczykiewicz| 60|7023.85|
|  8|  Zbigniew|              Pysla| 67|6176.82|
|  9|Mieczysław|Brzęczyszczykiewicz| 48|7611.33|
| 10|     Agata|       Mieczykowski| 26|8180.23|
+---+----------+-------------------+---+-------+
only showing top 10 rows

CPU times: user 5.08 ms, sys: 1.05 ms, total: 6.13 ms
Wall time: 126 ms


In [54]:
from pyspark.sql.functions import upper
def to_upper_str_columns(df, column_name, new_column_name):
    return df.withColumn(new_column_name, upper(df[column_name]))
df = df.transform(to_upper_str_columns, "firstname", "firstname_upper")

In [55]:
df.show(10)

+---+----------+-------------------+---+-------+---------------+
| id| firstname|           lastname|age| salary|firstname_upper|
+---+----------+-------------------+---+-------+---------------+
|  1| Krzysztof|       Mieczykowski| 31|7997.69|      KRZYSZTOF|
|  2|  Zbigniew|         Malinowski| 26|7493.97|       ZBIGNIEW|
|  3| Krzysztof|             Wlotka| 19|9745.33|      KRZYSZTOF|
|  4|     Agata|Brzęczyszczykiewicz| 43|7968.02|          AGATA|
|  5| Krzysztof|             Szczaw| 39|8192.67|      KRZYSZTOF|
|  6|   Wisława|               Glut| 35|8441.91|        WISŁAWA|
|  7|      Adam|Brzęczyszczykiewicz| 60|7023.85|           ADAM|
|  8|  Zbigniew|              Pysla| 67|6176.82|       ZBIGNIEW|
|  9|Mieczysław|Brzęczyszczykiewicz| 48|7611.33|     MIECZYSŁAW|
| 10|     Agata|       Mieczykowski| 26|8180.23|          AGATA|
+---+----------+-------------------+---+-------+---------------+
only showing top 10 rows



In [56]:
# filtrowanie numeryczne, ale tu na kolumnie typu str - czy jest poprawne?
df.filter(df["salary"] > 10000).count()

315830

In [57]:
# na ile partycji została nasza ramka danych rozrzucona po "klastrze"?
df.rdd.getNumPartitions()

28

In [58]:
%%time
# mierzymy czas operacji przy domyślnej liczbie partycji
df.filter(df["salary"] > 10000).count()

[Stage 56:==================>                                     (9 + 19) / 28]

CPU times: user 7.59 ms, sys: 3.1 ms, total: 10.7 ms
Wall time: 1.49 s


315830

In [59]:
df = df.repartition(12)

In [60]:
df.rdd.getNumPartitions()

[Stage 59:============================================>           (22 + 6) / 28]

12

In [61]:
%%time
# mierzymy czas operacji przy 12 partycjach dla 20_000_000 rekordów
df.filter(df["salary"] > 10000).count()

[Stage 60:=====================>                                 (11 + 17) / 28]

CPU times: user 8.83 ms, sys: 2.03 ms, total: 10.9 ms
Wall time: 1.68 s


315830

In [62]:
# macierz częstości dla dwóch kolumn - uwaga dla bardzo różnorodnych danych!
df.crosstab("firstname", "age").sort("firstname_age").show()

25/01/28 22:45:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 81:>                                                       (0 + 12) / 12]

+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|firstname_age|   18|   19|   20|   21|   22|   23|   24|   25|   26|   27|   28|   29|   30|   31|   32|   33|   34|   35|   36|   37|   38|   39|   40|   41|   42|   43|   44|   45|   46|   47|   48|   49|   50|   51|   52|   53|   54|   55|   56|   57|   58|   59|   60|   61|   62|   63|   64|   65|   66|   67|   68|
+-------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|         Adam|38957|39229|39021|3

In [63]:
# funkcja explain może przydać się w przypadku bardziej zaawansowanego debuggingu, optymalizacji i zrozumienia
# kolejności działania niektórych elementów silnika Spark
query = df.filter(df.firstname.contains('ski'))
query.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (5)
+- Exchange (4)
   +- Project (3)
      +- Filter (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [5]: [id#1534, firstname#1535, lastname#1536, age#1537, salary#1538]
Batched: false
Location: InMemoryFileIndex [file:/opt/spark/work-dir/employee.csv]
PushedFilters: [IsNotNull(firstname), StringContains(firstname,ski)]
ReadSchema: struct<id:int,firstname:string,lastname:string,age:int,salary:double>

(2) Filter
Input [5]: [id#1534, firstname#1535, lastname#1536, age#1537, salary#1538]
Condition : (isnotnull(firstname#1535) AND Contains(firstname#1535, ski))

(3) Project
Output [6]: [id#1534, firstname#1535, lastname#1536, age#1537, salary#1538, upper(firstname#1535) AS firstname_upper#1571]
Input [5]: [id#1534, firstname#1535, lastname#1536, age#1537, salary#1538]

(4) Exchange
Input [6]: [id#1534, firstname#1535, lastname#1536, age#1537, salary#1538, firstname_upper#1571]
Arguments: RoundRobinPartitioning(12), REPARTITION_BY_NUM, [plan_id=1

In [64]:
# zapisujemy ramkę do plików parquet
# zwróć uwagę na liczbę utworzonych plików

In [65]:
df.write.parquet('./data/parquet/')

25/01/28 22:45:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/28 22:45:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/28 22:45:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/28 22:45:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/28 22:45:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/01/28 22:45:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/28 22:45:15 WARN MemoryManager: Total allocation exceeds 95.

In [66]:
# lub chcąc nadpisać już istniejące dane - w trybie overwrite
df.write.mode("overwrite").parquet('./data/parquet/')

25/01/28 22:45:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/28 22:45:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/01/28 22:45:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/01/28 22:45:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/28 22:45:20 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/01/28 22:45:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/01/28 22:45:21 WARN MemoryManager: Total allocation exceeds 95.

In [68]:
from pyspark.sql import SparkSession

# Inicjalizacja SparkSession
spark = SparkSession.builder.appName("Recipe Reviews").getOrCreate()

# Wczytanie danych
data = spark.read.csv("data/Recipe Reviews and User Feedback Dataset.csv", header=True, inferSchema=True)

# Zmień nazwę kolumny
data = data.withColumnRenamed("_c0", "id")
data.show(5)

25/01/28 22:47:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
| id|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

25/01/28 22:47:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
 Schema: _c0, recipe_number, recipe_code, recipe_name, comment_id, user_id, user_name, user_reputation, created_at, reply_count, thumbs_up, thumbs_down, stars, best_score, text
Expected: _c0 but found: 
CSV file: file:///opt/spark/work-dir/data/Recipe%20Reviews%20and%20User%20Feedback%20Dataset.csv


In [69]:
data.select("reply_count") \
    .orderBy("reply_count", ascending=False) \
    .show(10)

+-----------+
|reply_count|
+-----------+
|          3|
|          3|
|          3|
|          3|
|          3|
|          3|
|          2|
|          2|
|          2|
|          2|
+-----------+
only showing top 10 rows



In [71]:
from pyspark.sql.functions import sum

data.groupBy("recipe_name") \
    .agg(sum("best_score").alias("total_best_score")) \
    .orderBy("total_best_score", ascending=False) \
    .show(10)

+--------------------+----------------+
|         recipe_name|total_best_score|
+--------------------+----------------+
|   Cheeseburger Soup|           98863|
|  Creamy White Chili|           85497|
|Amish Breakfast C...|           64880|
|Best Ever Banana ...|           64247|
|Favorite Chicken ...|           60755|
|Basic Homemade Bread|           59867|
|Flavorful Chicken...|           59195|
|Zucchini Pizza Ca...|           54032|
|Enchilada Casser-...|           51975|
|    Cauliflower Soup|           47905|
+--------------------+----------------+
only showing top 10 rows



In [74]:
from pyspark.sql.functions import count

data.groupBy("recipe_name") \
    .agg(count("reply_count").alias("total_comments")) \
    .orderBy("total_comments", ascending=False) \
    .show(10)

+--------------------+--------------+
|         recipe_name|total_comments|
+--------------------+--------------+
|   Cheeseburger Soup|           725|
|  Creamy White Chili|           654|
|Best Ever Banana ...|           509|
|Enchilada Casser-...|           421|
|Basic Homemade Bread|           397|
|Favorite Chicken ...|           395|
|Flavorful Chicken...|           368|
|Amish Breakfast C...|           338|
|Zucchini Pizza Ca...|           332|
|    Cauliflower Soup|           324|
+--------------------+--------------+
only showing top 10 rows



In [75]:
data.groupBy("stars") \
    .count() \
    .orderBy("stars") \
    .show()

+-----+-----+
|stars|count|
+-----+-----+
| NULL|   86|
|    0| 1696|
|    1|  280|
|    2|  232|
|    3|  490|
|    4| 1655|
|    5|13829|
+-----+-----+



In [76]:
employee_data = spark.read.csv("employee.csv", header=True, inferSchema=True)
employee_data.printSchema()
employee_data.show(5)

[Stage 112:================================>                     (17 + 11) / 28]

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)

+---+---------+-------------------+---+-------+
| id|firstname|           lastname|age| salary|
+---+---------+-------------------+---+-------+
|  1|Krzysztof|       Mieczykowski| 31|7997.69|
|  2| Zbigniew|         Malinowski| 26|7493.97|
|  3|Krzysztof|             Wlotka| 19|9745.33|
|  4|    Agata|Brzęczyszczykiewicz| 43|7968.02|
|  5|Krzysztof|             Szczaw| 39|8192.67|
+---+---------+-------------------+---+-------+
only showing top 5 rows



In [83]:
import time
string_employee_data = employee_data.withColumn("salary", employee_data["salary"].cast("string"))
start_time = time.time()

count = string_employee_data.filter(string_employee_data["salary"].cast("int") > 10000).count()

print(f"Czas wykonania operacji z typem string: {time.time() - start_time:.2f} sekundy")


[Stage 132:=============================================>         (23 + 5) / 28]

Czas wykonania operacji z typem string: 1.89 sekundy


In [84]:

start_time = time.time()

count = employee_data.filter(employee_data["salary"] > 10000).count()

print(f"Czas wykonania operacji z typem numeric: {time.time() - start_time:.2f} sekundy")


[Stage 135:===================================================>   (26 + 2) / 28]

Czas wykonania operacji z typem numeric: 1.48 sekundy


In [86]:
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.6 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [87]:
from pyspark.ml.feature import Bucketizer

splits = [10, 20, 30, 40, 50, 60, 70]

bucketizer = Bucketizer(
    splits=splits,
    inputCol="age",
    outputCol="age_bucket"
)

# Podziel dane na buckety
bucketed_data = bucketizer.transform(employee_data)

bucketed_data.select("age", "age_bucket").show(20)

+---+----------+
|age|age_bucket|
+---+----------+
| 31|       2.0|
| 26|       1.0|
| 19|       0.0|
| 43|       3.0|
| 39|       2.0|
| 35|       2.0|
| 60|       5.0|
| 67|       5.0|
| 48|       3.0|
| 26|       1.0|
| 23|       1.0|
| 33|       2.0|
| 51|       4.0|
| 47|       3.0|
| 65|       5.0|
| 64|       5.0|
| 21|       1.0|
| 48|       3.0|
| 21|       1.0|
| 50|       4.0|
+---+----------+
only showing top 20 rows

